+++++[link text](https:// [link text](https://))### 1. Read environment

In [1]:
import os

current_directory = os.getcwd()
print("Current Directory:", current_directory)

Current Directory: C:\Users\uddin81\Documents\Customize_RL_MEC Env\Test for Journal\Final Result\backup\10k20k_8e9_300m__50_3_170_200


In [2]:
# ! pip install fastai

In [3]:
import numpy as np
from mec_environment import MECEnvironment
from random_agent import RandomAgent

mec_evn = MECEnvironment(user_cnt=50, mec_cnt=3,Env_Type=1)
print(mec_evn.mec_servers)

194
{}
98
70
26
{'data_size': 18171, 'circle': 8384407, 'user_index': 48, 'priority': 1, 'user_distance': [95.27442286803297, 205.49528914373454, 505.35078120815973], 'delta_max': 0.192}
{'data_size': 12371, 'circle': 9482442, 'user_index': 49, 'priority': 2, 'user_distance': [276.4697081413863, 25.733657589615518, 323.86562452056535], 'delta_max': 0.186}
{'data_size': 12915, 'circle': 9242029, 'user_index': 29, 'priority': 3, 'user_distance': [528.6595715645917, 228.78363586343124, 72.1315566927862], 'delta_max': 0.179}
[<mec_server.MECServer object at 0x00000208D1A24190>, <mec_server.MECServer object at 0x00000208D1E4E290>, <mec_server.MECServer object at 0x00000208D1E4E790>]


### 2. Examine the State and Action Spaces

In [4]:
action_size = mec_evn.action_size()

task=mec_evn.get_task()

states = mec_evn.get_state(task)
print(mec_evn.trans_speed_matrix)
state_size = mec_evn.state_size()
action_space = mec_evn.actions
print('Action size: {}'.format(action_size))
print('Action space: {}'.format(action_space))
print('Observes a state with length: {}'.format(state_size))
print('state size: {}'.format(len(states)))
print('The state for the first agent looks like:', states)
print('max_freq',mec_evn.max_freq_server)

{48: {'speed': array([526451.79595463,   8230.95217677,   1303.74710805])}, 49: {'speed': array([   8503.4226393 , 3558903.9784338 ,    5513.09329017])}, 29: {'speed': array([2.02426928e+02, 5.33731169e+03, 5.81458252e+05])}}
Action size: 45
Action space: [(0, 0.1, 0), (0, 0.1, 1), (0, 0.1, 2), (0, 0.325, 0), (0, 0.325, 1), (0, 0.325, 2), (0, 0.55, 0), (0, 0.55, 1), (0, 0.55, 2), (0, 0.775, 0), (0, 0.775, 1), (0, 0.775, 2), (0, 1.0, 0), (0, 1.0, 1), (0, 1.0, 2), (1, 0.1, 0), (1, 0.1, 1), (1, 0.1, 2), (1, 0.325, 0), (1, 0.325, 1), (1, 0.325, 2), (1, 0.55, 0), (1, 0.55, 1), (1, 0.55, 2), (1, 0.775, 0), (1, 0.775, 1), (1, 0.775, 2), (1, 1.0, 0), (1, 1.0, 1), (1, 1.0, 2), (2, 0.1, 0), (2, 0.1, 1), (2, 0.1, 2), (2, 0.325, 0), (2, 0.325, 1), (2, 0.325, 2), (2, 0.55, 0), (2, 0.55, 1), (2, 0.55, 2), (2, 0.775, 0), (2, 0.775, 1), (2, 0.775, 2), (2, 1.0, 0), (2, 1.0, 1), (2, 1.0, 2)]
Observes a state with length: 27
state size: 27
The state for the first agent looks like: [1.44457217e-01 2.33332

### 3. Take Random Actions in the Environment

### 6. Take Actions with DQN

In [5]:
epoch_no =3000
window_size =250

In [6]:
import torch
from collections import deque

import matplotlib.pyplot as plt

%matplotlib inline

In [7]:
from dqn_agent import Agent
agent = Agent(state_size=state_size,action_size=action_size,seed=0)
model_path = os.path.join(current_directory, 'model_dqnp_checkpoint.pth')


In [8]:
import math
Mec_2_Mec=300
Range=(Mec_2_Mec/2)

max_dis=math.sqrt(10*10+Range*Range)
R_1=(max_dis-10)/4


bucket_ranges = [(10, 10+R_1) ,(10+R_1,10+2*R_1),(10+2*R_1,10+3*R_1),(10+3*R_1,10+4*R_1),(10+4*R_1,float('inf'))]

print(bucket_ranges)
# Function to determine the bucket for a given value
def find_bucket(value, ranges):
    for start, end in ranges:
        if start <= value <= end:
            return (start, end)
    return None

[(10, 45.08324094593227), (45.08324094593227, 80.16648189186454), (80.16648189186454, 115.24972283779681), (115.24972283779681, 150.33296378372907), (150.33296378372907, inf)]


In [ ]:
def dqnp(n_episodes=epoch_no, max_t=1000, eps_start=1.0, eps_end=0.001, eps_decay=0.9975,bench_score=1500000):
    """Deep Q-Learning.

    Params
    ======
        n_episodes (int): maximum number of training episodes
        max_t (int): maximum number of atimesteps per episode
        eps_start (float): starting value of epsilon, for epsilon-greedy action selection
        eps_end (float): minimum value of epsilon
        eps_decay (float): multiplicative factor (per episode) for decreasing epsilon
    """
    epi_scores = []                        # list containing scores from each episode
    epi_task=[]
    epi_task_P1=[]
    epi_task_P2=[]
    epi_task_P3=[]
    
    epi_all_trans_latency=[]
    epi_all_trans_latency_P1=[]
    epi_all_trans_latency_P2=[]
    epi_all_trans_latency_P3=[]
    epi_all_comp_latency=[]
    epi_all_comp_latency_P1=[]
    epi_all_comp_latency_P2=[]
    epi_all_comp_latency_P3=[]
    
    
    epi_comp_latency=[]
    epi_comp_latency_P1=[]
    epi_comp_latency_P2=[]
    epi_comp_latency_P3=[]
    epi_tran_latency=[]
    epi_tran_latency_P1=[]
    epi_tran_latency_P2=[]
    epi_tran_latency_P3=[]
    epi_latency=[]
    epi_latency_P1=[]
    epi_latency_P2=[]
    epi_latency_P3=[]
    epi_task_select_P1=[]
    epi_task_select_P2=[]
    epi_task_select_P3=[]
    epi_datasize=[]
    epi_datasize_P1=[]
    epi_datasize_P2=[]
    epi_datasize_P3=[]
    epi_all_datasize_P1=[]
    epi_all_datasize_P2=[]
    epi_all_datasize_P3=[]
    epi_energy=[]
    epi_energy_P1=[]
    epi_energy_P2=[]
    epi_energy_P3=[]

    epi_num_task=[]
    epi_num_task_P1=[]
    epi_num_task_P2=[]
    epi_num_task_P3=[]
    epi_comp_energy=[]
    epi_comp_frequency=[]
    mean_scores = [] # average score by the windos size
    mean_task=[] # average task ratio by window size
    mean_energy = [] # average energy cost by window size
    mean_latency=[]
    mean_datasize=[]
    mean_tran_latency=[]
    mean_comp_latency=[]

    mean_tasks_P1=[]
    mean_tasks_P2=[]
    mean_tasks_P3=[]

    mean_energy_P1=[]
    mean_energy_P2=[]
    mean_energy_P3=[]

    mean_latency_P1=[]
    mean_latency_P2=[]
    mean_latency_P3=[]
    mean_tran_latency_P1=[]
    mean_tran_latency_P2=[]
    mean_tran_latency_P3=[]
    mean_comp_latency_P1=[]
    mean_comp_latency_P2=[]
    mean_comp_latency_P3=[]

    mean_datasize_P1=[]
    mean_datasize_P2=[]
    mean_datasize_P3=[]
    reward_list_P1=[]
    reward_list_P2=[]
    reward_list_P3=[]

    scores_window = deque(maxlen=window_size)  # last score_window_size scores
    task_done_ratio_window = deque(maxlen=window_size)  # last  task dones
    energy_window = deque(maxlen=window_size)  # last  engery cost
    latency_window = deque(maxlen=window_size)  # last  engery cost
    datasize_window= deque(maxlen=window_size)


    mean_tasks_P1_window = deque(maxlen=window_size)  # last  engery cost
    mean_tasks_P2_window = deque(maxlen=window_size)  # last  engery cost
    mean_tasks_P3_window = deque(maxlen=window_size)  # last  engery cost

    win_num_task = deque(maxlen=window_size)
    win_num_task_P1 = deque(maxlen=window_size)  # last  engery cost
    win_num_task_P2 = deque(maxlen=window_size)  # last  engery cost
    win_num_task_P3 = deque(maxlen=window_size)  # last  engery cost

    mean_datasize_window=deque(maxlen=window_size)
    mean_datasize_P1_window= deque(maxlen=window_size)  # last  engery cost
    mean_datasize_P2_window = deque(maxlen=window_size)  # last  engery cost
    mean_datasize_P3_window = deque(maxlen=window_size)  # last  engery cost

    mean_energy_window_P1 = deque(maxlen=window_size)  # last  engery cost
    mean_energy_window_P2 = deque(maxlen=window_size)  # last  engery cost
    mean_energy_window_P3 = deque(maxlen=window_size)  # last  engery cost

    mean_latency_P1_window=deque(maxlen=window_size)
    mean_latency_P2_window=deque(maxlen=window_size)
    mean_latency_P3_window=deque(maxlen=window_size)
    
    mean_comp_latency_P1_window=deque(maxlen=window_size)
    mean_comp_latency_P2_window=deque(maxlen=window_size)
    mean_comp_latency_P3_window=deque(maxlen=window_size)
    
    mean_tran_latency_P1_window=deque(maxlen=window_size)
    mean_tran_latency_P2_window=deque(maxlen=window_size)
    mean_tran_latency_P3_window=deque(maxlen=window_size)
        
    
    
    (10, 10+R_1) ,(10+R_1,10+2*R_1),(10+2*R_1,10+3*R_1),(10+3*R_1,10+4*R_1),(10+4*R_1,float('inf'))
    
    bucket_tasks_mean = {(10, 10+R_1):{},(10+R_1,10+2*R_1):{},(10+2*R_1,10+3*R_1):{},\
                         (10+3*R_1,10+4*R_1):{},(10+4*R_1,float('inf')):{}}
    bucket_task_len= {(10, 10+R_1):{},(10+R_1,10+2*R_1):{},(10+2*R_1,10+3*R_1):{},\
                         (10+3*R_1,10+4*R_1):{},(10+4*R_1,float('inf')):{}}
    bucket_latency_mean={(10, 10+R_1):{},(10+R_1,10+2*R_1):{},(10+2*R_1,10+3*R_1):{},\
                         (10+3*R_1,10+4*R_1):{},(10+4*R_1,float('inf')):{}}
    bucket_energy_mean= {(10, 10+R_1):{},(10+R_1,10+2*R_1):{},(10+2*R_1,10+3*R_1):{},\
                         (10+3*R_1,10+4*R_1):{},(10+4*R_1,float('inf')):{}}
    bucket_data_mean= {(10, 10+R_1):{},(10+R_1,10+2*R_1):{},(10+2*R_1,10+3*R_1):{},\
                         (10+3*R_1,10+4*R_1):{},(10+4*R_1,float('inf')):{}}
    #bucket_energy_mean={(10, 0.25*max_dis):{},(0.25*max_dis,0.50*max_dis):{},(0.5*max_dis,0.75*max_dis):{},(0.75*max_dis,1*max_dis):{},(1*max_dis,float('inf')):{}}
    eps = eps_start                    # initialize epsilon
    for i_episode in range(1, n_episodes+1):
        mec_evn.reset() # reset the environment
        #mec_evn.max_freq_server= [np.random.choice([2,4,8,16])*1e9 for _ in range (3)]
       # print(mec_evn.max_freq_server)
     #   print(mec_evn.mec_servers)

        score = 0
        energy = []
        latency=[]
        task_done = []
        energy_P1=[]
        energy_P2=[]
        energy_P3=[]
        task_done_P1=[]
        task_done_P2=[]
        task_done_P3=[]
        latency_P1=[]
        latency_P2=[]
        latency_P3=[]
        comp_latency=[]
        comp_latency_P1=[]
        comp_latency_P2=[]
        comp_latency_P3=[]
        tran_latency=[]
        tran_latency_P1=[]
        tran_latency_P2=[]
        tran_latency_P3=[]
        compt_energy=[]
        compt_freq=[]
        all_trans_latency=[]
        all_trans_latency_P1=[]
        all_trans_latency_P2=[]
        all_trans_latency_P3=[]
        
        all_comp_latency=[]
        all_comp_latency_P1=[]
        all_comp_latency_P2=[]
        all_comp_latency_P3=[]
        
        all_datasize_P1=[]
        all_datasize_P2=[]
        all_datasize_P3=[]
        datasize=[]
        datasize_P1=[]
        datasize_P2=[]
        datasize_P3=[]

        reward_lat=[]
        reward_eng=[]
        bucket_task = {range_tuple: {'P1_task_done': [], 'P2_task_done': [], 'P3_task_done': []} for range_tuple in bucket_ranges}
        bucket_latency = {range_tuple: {'P1_latency': [], 'P2_latency': [], 'P3_latency': []} for range_tuple in bucket_ranges}
        bucket_energy = {range_tuple: {'P1_energy': [], 'P2_energy': [], 'P3_energy': []} for range_tuple in bucket_ranges}
        bucket_data = {range_tuple: {'P1_data': [], 'P2_data': [], 'P3_data': []} for range_tuple in bucket_ranges}


        tasks = mec_evn.get_task()
        state = mec_evn.get_state(tasks)
        for t in range(max_t):

            
            action = agent.act(state, eps)
            next_state, feedback,done,action_task,next_task,rec_mec_idx = mec_evn.step(action,tasks)
            agent.step(state, action, feedback['reward'], next_state, done)
           
            #print(mec_evn.max_freq_server)

            score += feedback['reward']
            energy.append(feedback['energy'])
            
            user_distance=action_task['user_distance'][rec_mec_idx]
            #print(user_distance)
           # print(user_distance,rec_mec_idx,action_task['user_distance'])
            bucket = find_bucket(user_distance, bucket_ranges)
            tran_time=feedback['lat_vec'][0]
            comp_time=feedback['lat_vec'][1]+feedback['lat_vec'][2] #comp_time+wait_time
            reward_lat.append(feedback['reward_lat'])
            reward_eng.append(feedback['reward_eng'])
            all_trans_latency.append(tran_time)
            all_comp_latency.append(comp_time)
            trans_energy=feedback['eng_vector'][0]
            comp_energy=feedback['eng_vector'][1]
            compt_energy.append(comp_energy)
            compt_freq.append(max(2*10**9,mec_evn.max_freq_server[rec_mec_idx]*mec_evn.actions[action][1]))
            
           # print(min(2*10**9,mec_evn.max_freq_server[rec_mec_idx]*mec_evn.actions[action][1]))
           # print(tran_time,comp_time)
            if feedback['task_done']==1:
              datasize.append(action_task['data_size'])
              latency.append(feedback['latency'])
              comp_latency.append(comp_time)
              tran_latency.append(tran_time)


            if action_task['data_size'] != 0 :
              task_done.append(feedback['task_done'])
              
              if action_task['priority']==1:
                  
                  task_done_P1.append(feedback['task_done'])
                  energy_P1.append(feedback['energy'])
                  
                  
                  bucket_task[bucket]['P1_task_done'].append(feedback['task_done'])
                  bucket_energy[bucket]['P1_energy'].append(feedback['energy'])
                  all_datasize_P1.append(action_task['data_size'])
                  
                  all_trans_latency_P1.append(tran_time)
                  all_comp_latency_P1.append(comp_time)
                  if feedback['task_done']:
                    latency_P1.append(feedback['latency'])
                    comp_latency_P1.append(comp_time)
                    tran_latency_P1.append(tran_time)
                    datasize_P1.append(action_task['data_size'])
                    
                    bucket_latency[bucket]['P1_latency'].append( tran_time)
                    bucket_data[bucket]['P1_data'].append(action_task['data_size'])
                  
                    
                    
                    
                    
                    

              if action_task['priority']==2:
                  
                  task_done_P2.append(feedback['task_done'])
                  energy_P2.append(feedback['energy'])
                  
                  
                  bucket_task[bucket]['P2_task_done'].append(feedback['task_done'])
                  bucket_energy[bucket]['P2_energy'].append(feedback['energy'])
                  
                  all_datasize_P2.append(action_task['data_size'])
                  all_trans_latency_P2.append(tran_time)
                  all_comp_latency_P2.append(comp_time)
                 
                  if feedback['task_done']:
                    latency_P2.append(feedback['latency'])
                    comp_latency_P2.append(comp_time)
                    tran_latency_P2.append(tran_time)
                    datasize_P2.append(action_task['data_size'])
                    
                    bucket_latency[bucket]['P2_latency'].append(tran_time)
                    bucket_data[bucket]['P2_data'].append(action_task['data_size'])
                    
                    
                    
                    

              if action_task['priority']==3:
                 
                  task_done_P3.append(feedback['task_done'])
                  energy_P3.append(feedback['energy'])
                  
                  
                  bucket_task[bucket]['P3_task_done'].append(feedback['task_done'])
                  bucket_energy[bucket]['P3_energy'].append(feedback['energy'])
                  all_datasize_P3.append(action_task['data_size'])
                  all_trans_latency_P3.append(tran_time)
                  all_comp_latency_P3.append(comp_time)
                 
                  if feedback['task_done']:
                    latency_P3.append(feedback['latency'])
                    comp_latency_P3.append(comp_time)
                    tran_latency_P3.append(tran_time)
                    datasize_P3.append(action_task['data_size'])
                    
                    bucket_latency[bucket]['P3_latency'].append(tran_time)
                    bucket_data[bucket]['P3_data'].append(action_task['data_size'])
                    
                    
                   

            

            
        
                # Append values to their respective lists within the bucket
                
               



            
            tasks=next_task
            state=next_state
            if done:
                print(t)
                
                break
        

        for range_tuple, tasks in bucket_task.items():
            
            
            # Initialize a sub-dictionary for each range
            for task, values in tasks.items():
        # Calculate the mean for each task list
                mean_value = sum(values) / len(values) if values else 0  # Ensure division is valid
                mean_len = len(values)
                # Create a new key for storing the mean value of the task list
                mean_key = f'{task}_mean'
                len_key = f'{task}_mean_len'
                # Ensure the sub-dictionary for mean values exists for the current range_tuple
                if mean_key not in bucket_tasks_mean[range_tuple]:
                    bucket_tasks_mean[range_tuple][mean_key] = []
                if len_key not in bucket_task_len[range_tuple]:
                    bucket_task_len[range_tuple][len_key] = []
                # Store the mean value in the corresponding range's sub-dictionary
                bucket_tasks_mean[range_tuple][mean_key].append(mean_value)
                bucket_task_len[range_tuple][len_key].append(mean_len)
        for range_tuple, lat in bucket_latency.items():
             
            
            # Initialize a sub-dictionary for each range
            for l, v in lat.items():
               # print(l,v)
                # Calculate the mean for each task list
                mean_value = sum(v) / len(v) if v else 0  # Ensure division is valid
                
                # Create a new key for storing the mean value of the task list
                mean_key = f'{l}_mean'
                if mean_key not in bucket_latency_mean[range_tuple]:
                    bucket_latency_mean[range_tuple][mean_key] = []
                
                # Store the mean value in the corresponding range's sub-dictionary
                bucket_latency_mean[range_tuple][mean_key].append(mean_value)
        for range_tuple, eng in bucket_energy.items():
             
            
            # Initialize a sub-dictionary for each range
            for e, v in eng.items():
                # Calculate the mean for each task list
                mean_value = sum(v) / len(v) if v else 0  # Ensure division is valid
                
                # Create a new key for storing the mean value of the task list
                mean_key = f'{e}_mean'
                if mean_key not in bucket_energy_mean[range_tuple]:
                    bucket_energy_mean[range_tuple][mean_key] = []
                # Store the mean value in the corresponding range's sub-dictionary
                bucket_energy_mean[range_tuple][mean_key].append(mean_value)
        
        for range_tuple, dat in bucket_data.items():
             
            
            # Initialize a sub-dictionary for each range
            for d, v in dat.items():
                # Calculate the mean for each task list
                mean_value = sum(v) / len(v) if v else 0  # Ensure division is valid
                
                # Create a new key for storing the mean value of the task list
                mean_key = f'{d}_mean'
                if mean_key not in bucket_data_mean[range_tuple]:
                    bucket_data_mean[range_tuple][mean_key] = []
                # Store the mean value in the corresponding range's sub-dictionary
                bucket_data_mean[range_tuple][mean_key].append(mean_value)
                

# Now, bucket_means contains only the mean values for each task list within each bucket range.
# Displaying the new bucket_means dictionary


        scores_window.append(score)       # save most recent score
        epi_scores.append(score)              # save most recent score
        mean_score = np.mean(scores_window)
        epi_task.append(np.mean(task_done))
        epi_task_P1.append(np.mean(task_done_P1))
        epi_task_P2.append(np.mean(task_done_P2))
        epi_task_P3.append(np.mean(task_done_P3))

        epi_task_select_P1.append(len(task_done_P1))
        epi_task_select_P2.append(len(task_done_P2))
        epi_task_select_P3.append(len(task_done_P3))

        epi_latency.append(np.mean(latency))
        epi_latency_P1.append(np.mean(latency_P1))
        epi_latency_P2.append(np.mean(latency_P2))
        epi_latency_P3.append(np.mean(latency_P3))
        
        epi_comp_latency.append(np.mean(comp_latency))
        epi_comp_latency_P1.append(np.mean(comp_latency_P1))
        epi_comp_latency_P2.append(np.mean(comp_latency_P2))
        epi_comp_latency_P3.append(np.mean(comp_latency_P3))
        
        epi_tran_latency.append(np.mean(tran_latency))
        epi_tran_latency_P1.append(np.mean(tran_latency_P1))
        epi_tran_latency_P2.append(np.mean(tran_latency_P2))
        epi_tran_latency_P3.append(np.mean(tran_latency_P3))
        

        epi_energy.append(np.mean(energy))
        epi_energy_P1.append(np.mean(energy_P1))
        epi_energy_P2.append(np.mean(energy_P2))
        epi_energy_P3.append(np.mean(energy_P3))

        epi_num_task.append(sum(task_done_P1)+sum(task_done_P2)+sum(task_done_P3))
        epi_num_task_P1.append(sum(task_done_P1))
        epi_num_task_P2.append(sum(task_done_P2))
        epi_num_task_P3.append(sum(task_done_P3))

        epi_datasize.append(np.mean(datasize))
        epi_datasize_P1.append(np.mean(datasize_P1))
        epi_datasize_P2.append(np.mean(datasize_P2))
        epi_datasize_P3.append(np.mean(datasize_P3))
        
        epi_all_datasize_P1.append(np.mean(all_datasize_P1))
        epi_all_datasize_P2.append(np.mean(all_datasize_P2))
        epi_all_datasize_P3.append(np.mean(all_datasize_P3))

        
        epi_all_trans_latency.append(np.mean(all_trans_latency))
        epi_all_trans_latency_P1.append(np.mean(all_trans_latency_P1))
        epi_all_trans_latency_P2.append(np.mean(all_trans_latency_P2))
        epi_all_trans_latency_P3.append(np.mean(all_trans_latency_P3))
        epi_all_comp_latency.append(np.mean(all_comp_latency))
        epi_all_comp_latency_P1.append(np.mean(all_comp_latency_P1))
        epi_all_comp_latency_P2.append(np.mean(all_comp_latency_P2))
        epi_all_comp_latency_P3.append(np.mean(all_comp_latency_P3))
       
        epi_comp_energy.append(np.mean(compt_energy))
        epi_comp_frequency.append(np.mean(compt_freq))

        eps = max(eps_end, eps_decay*eps) # decrease epsilon

        print('\rEpisode {}\tAverage Score: {:.4f}\tmeantaskP1: {:.4f}\tmeantaskP2: {:.4f}\tmeantaskP3: {:.4f}\tcomplatencyP1: \
            {:.4f}\tcomplatencyP2: {:.4f}\tcomplatencyP3: {:.4f}\tmeanlatencyP1: {:.4f}\tmeanlatencyP2: {:.4f}\tmeanlatencyP3:{:.4f}\tmeanlatreward:{:.4f}\tmeanengreward:{:.4f}'.format(i_episode, \
            mean_score, np.mean(task_done_P1), np.mean(task_done_P2), np.mean(task_done_P3), np.mean(comp_latency_P1), \
            np.mean(comp_latency_P2), np.mean(comp_latency_P3), \
                                    np.mean(tran_latency_P1), np.mean(tran_latency_P2), np.mean(tran_latency_P3),np.mean(reward_lat),np.mean(reward_eng)), end="")





        if i_episode % window_size == 0:
            

            mean_scores.append(mean_score)
            task_done_ratio_window.append(np.mean(task_done))       # task done ratio
            mean_task.append(np.mean(task_done_ratio_window))
            energy_window.append(np.mean(energy))
            latency_window.append(np.mean(latency))

            mean_energy.append(np.mean(energy_window))
            mean_latency.append(np.mean(latency_window))


            mean_datasize_window.append(np.mean(datasize))
            mean_datasize.append(np.mean(mean_datasize_window))

            mean_tasks_P1_window.append(np.mean(task_done_P1))
            mean_tasks_P1.append(np.mean( mean_tasks_P1_window))

            mean_tasks_P2_window.append(np.mean(task_done_P2))
            mean_tasks_P2.append(np.mean( mean_tasks_P2_window))

            mean_tasks_P3_window.append(np.mean(task_done_P3))
            mean_tasks_P3.append(np.mean( mean_tasks_P3_window))

            mean_energy_window_P1.append(np.mean(energy_P1))
            mean_energy_P1.append(np.mean(mean_energy_window_P1))

            mean_energy_window_P2.append(np.mean(energy_P2))
            mean_energy_P2.append(np.mean(mean_energy_window_P2))

            mean_energy_window_P3.append(np.mean(energy_P3))
            mean_energy_P3.append(np.mean(mean_energy_window_P3))


            mean_latency_P1_window.append(np.mean(latency_P1))
            mean_latency_P1.append(np.mean( mean_latency_P1_window))

            mean_latency_P2_window.append(np.mean(latency_P2))
            mean_latency_P2.append(np.mean( mean_latency_P2_window))

            mean_latency_P3_window.append(np.mean(latency_P3))
            mean_latency_P3.append(np.mean( mean_latency_P3_window))
            
            mean_comp_latency_P1_window.append(np.mean(comp_latency_P1))
            mean_comp_latency_P1.append(np.mean( mean_comp_latency_P1_window))
            mean_comp_latency_P2_window.append(np.mean(comp_latency_P2))
            mean_comp_latency_P2.append(np.mean( mean_comp_latency_P2_window))
            mean_comp_latency_P3_window.append(np.mean(comp_latency_P3))
            mean_comp_latency_P3.append(np.mean( mean_comp_latency_P3_window))
            
            mean_tran_latency_P1_window.append(np.mean(tran_latency_P1))
            mean_tran_latency_P1.append(np.mean( mean_tran_latency_P1_window))
            mean_tran_latency_P2_window.append(np.mean(tran_latency_P2))
            mean_tran_latency_P2.append(np.mean( mean_tran_latency_P2_window))
            mean_tran_latency_P3_window.append(np.mean(tran_latency_P3))
            mean_tran_latency_P3.append(np.mean( mean_tran_latency_P3_window))

            mean_datasize_P1_window.append(np.mean(datasize_P1))
            mean_datasize_P1.append(np.mean(mean_datasize_P1_window))

            mean_datasize_P2_window.append(np.mean(datasize_P2))
            mean_datasize_P2.append(np.mean(mean_datasize_P2_window))

            mean_datasize_P3_window.append(np.mean(datasize_P3))
            mean_datasize_P3.append(np.mean(mean_datasize_P3_window))


            win_num_task.append(sum(task_done_P1)+sum(task_done_P2)+sum(task_done_P3))
            win_num_task_P1.append(sum(task_done_P1))
            win_num_task_P2.append(sum(task_done_P2))
            win_num_task_P3.append(sum(task_done_P3))


            print('\rEpisode {}\tAverage Score: {:.4f}\tmeantaskP1: {:.4f}\tmeantaskP2: {:.4f}\tmeantaskP3: {:.4f}\tmeanenergyP1: \
    {:.4f}\tmeanenergyP2: {:.4f}\tmeanenergyP3: {:.4f} \tmeanlatencyP1: {:.4f} \tmeanlatencyP2: {:.4f} \tmeanlatencyP3: \
    {:.4f} \tmeandatasize: {:.4f} \tmeandatasizeP1: {:.4f} \tmeandatasizeP2: {:.4f} \tmeandatasizeP3: {:.4f} \tmean_tran_latencyP1: {:.4f}  \
    \tmean_tran_latencyP2: {:.4f}  \tmean_tran_latencyP3: {:.4f}  \tmean_comp_latencyP1: {:.4f}  \
    \tmean_comp_latencyP2: {:.4f} \tmean_comp_latencyP3: {:.4f}'.format(
        i_episode, mean_score, np.mean(mean_tasks_P1[-window_size:]), np.mean(mean_tasks_P2[-window_size:]),
        np.mean(mean_tasks_P3[-window_size:]), np.mean(mean_energy_P1[-window_size:]), np.mean(mean_energy_P2[-window_size:]),
        np.mean(mean_energy_P3[-window_size:]), np.mean(mean_latency_P1[-window_size:]), np.mean(mean_latency_P2[-window_size:]),
        np.mean(mean_latency_P3[-window_size:]), np.mean(mean_datasize[-window_size:]), np.mean(mean_datasize_P1[-window_size:]),
        np.mean(mean_datasize_P2[-window_size:]), np.mean(mean_datasize_P3[-window_size:]), np.mean(mean_tran_latency_P1[-window_size:]),
        np.mean(mean_tran_latency_P2[-window_size:]), np.mean(mean_tran_latency_P3[-window_size:]), np.mean(mean_comp_latency_P1[-window_size:]),
        np.mean(mean_comp_latency_P2[-window_size:]), np.mean(mean_comp_latency_P3[-window_size:]))
)
                                                                                           

            #print(len(task_done_P1),len(task_done_P2),len(task_done_P3))
           # print(len(task_done_P1)+len(task_done_P2)+len(task_done_P3))
            print('mean_task_till_this_window',np.mean(epi_task[-window_size:]), 'T1::',   np.mean(epi_task_P1[-window_size:]) ,'T2::' ,np.mean(epi_task_P2[-window_size:]) ,'T3::',np.mean(epi_task_P3[-window_size:]))
            #print('mean_energy_till_this_window',np.mean(mean_energy[-window_size:]), 'T1::', np.mean(mean_energy_P1[-window_size:])  ,'T2::', np.mean(mean_energy_P2[-window_size:])  ,'T3::',np.mean(mean_energy_P3[-window_size:]))
           # print('mean Latency till this window',np.mean(mean_latency[-window_size:]),'T1::', np.mean(mean_latency_P1[-window_size:]),'T1::', np.mean(mean_latency_P2[-window_size:]),'T1::', np.mean(mean_latency_P3[-window_size:]))
           # print(type(mean_tasks_P1),type(epi_num_task_P1))
            #print('mean_datasize', mean_datasize[-1])
            print( 'task_select_P1', np.mean(epi_task_select_P1[-window_size:]), 'task_select_P2', np.mean(epi_task_select_P2[-window_size:]),'task_select_P3', np.mean(epi_task_select_P3[-window_size:]))
            print('total_task_done_P1::', np.mean(epi_num_task_P1[-window_size:]),'total_task_done_P2::', np.mean(epi_num_task_P2[-window_size:]),'total_task_done_P3::',np.mean(epi_num_task_P3[-window_size:]))

            #print(np.mean(reward_list_P1), np.mean(reward_list_P2), np.mean(reward_list_P3))
            #print(min(reward_list_P1), min(reward_list_P2), min(reward_list_P3))
           # print(max(reward_list_P1), max(reward_list_P2), max(reward_list_P3))
            
            #print(mean_tasks_P1)
           # print(epi_task_P1)
            print('mean_task_P1_ratio_by_bucket_51_10_100', np.mean(bucket_tasks_mean[(10,10+R_1)]['P1_task_done_mean'][-window_size:]),
                  np.mean(bucket_tasks_mean[(10+R_1,10+2*R_1)]['P1_task_done_mean'][-window_size:]) , np.mean(bucket_tasks_mean[(10+2*R_1,10+3*R_1)]['P1_task_done_mean'][-window_size:])
                  ,np.mean(bucket_tasks_mean[(10+3*R_1,10+4*R_1)]['P1_task_done_mean'][-window_size:]), np.mean(bucket_tasks_mean[(10+4*R_1,float('inf'))]['P1_task_done_mean'][-window_size:]))
            print('mean_task_P2_ratio_by_bucket_51_10_100', np.mean(bucket_tasks_mean[(10,10+R_1)]['P2_task_done_mean'][-window_size:]),
                  np.mean(bucket_tasks_mean[(10+R_1,10+2*R_1)]['P2_task_done_mean'][-window_size:]) , np.mean(bucket_tasks_mean[(10+2*R_1,10+3*R_1)]['P2_task_done_mean'][-window_size:])
                  ,np.mean(bucket_tasks_mean[(10+3*R_1,10+4*R_1)]['P2_task_done_mean'][-window_size:]), np.mean(bucket_tasks_mean[(10+4*R_1,float('inf'))]['P2_task_done_mean'][-window_size:]))
            print('mean_task_P3_ratio_by_bucket_51_10_100', np.mean(bucket_tasks_mean[(10,10+R_1)]['P3_task_done_mean'][-window_size:]),
                  np.mean(bucket_tasks_mean[(10+R_1,10+2*R_1)]['P3_task_done_mean'][-window_size:]) , np.mean(bucket_tasks_mean[(10+2*R_1,10+3*R_1)]['P3_task_done_mean'][-window_size:])
                  ,np.mean(bucket_tasks_mean[(10+3*R_1,10+4*R_1)]['P3_task_done_mean'][-window_size:]), np.mean(bucket_tasks_mean[(10+4*R_1,float('inf'))]['P3_task_done_mean'][-window_size:]))
            print('mean_task_P1_select_by_bucket_51_10_100', np.mean(bucket_task_len[(10,10+R_1)]['P1_task_done_mean_len'][-window_size:]),
                  np.mean(bucket_task_len[(10+R_1,10+2*R_1)]['P1_task_done_mean_len'][-window_size:]) , np.mean(bucket_task_len[(10+2*R_1,10+3*R_1)]['P1_task_done_mean_len'][-window_size:])
                  ,np.mean(bucket_task_len[(10+3*R_1,10+4*R_1)]['P1_task_done_mean_len'][-window_size:]), np.mean(bucket_task_len[(10+4*R_1,float('inf'))]['P1_task_done_mean_len'][-window_size:]))
            print('mean_task_P2_select_by_bucket_51_10_100', np.mean(bucket_task_len[(10,10+R_1)]['P2_task_done_mean_len'][-window_size:]),
                  np.mean(bucket_task_len[(10+R_1,10+2*R_1)]['P2_task_done_mean_len'][-window_size:]) , np.mean(bucket_task_len[(10+2*R_1,10+3*R_1)]['P2_task_done_mean_len'][-window_size:])
                  ,np.mean(bucket_task_len[(10+3*R_1,10+4*R_1)]['P2_task_done_mean_len'][-window_size:]), np.mean(bucket_task_len[(10+4*R_1,float('inf'))]['P2_task_done_mean_len'][-window_size:]))
            print('mean_task_P3_select_by_bucket_51_10_100', np.mean(bucket_task_len[(10,10+R_1)]['P3_task_done_mean_len'][-window_size:]),
                  np.mean(bucket_task_len[(10+R_1,10+2*R_1)]['P3_task_done_mean_len'][-window_size:]) , np.mean(bucket_task_len[(10+2*R_1,10+3*R_1)]['P3_task_done_mean_len'][-window_size:])
                  ,np.mean(bucket_task_len[(10+3*R_1,10+4*R_1)]['P3_task_done_mean_len'][-window_size:]), np.mean(bucket_task_len[(10+4*R_1,float('inf'))]['P3_task_done_mean_len'][-window_size:]))

            print('mean_task_P1_latency_by_bucket_51_10_100', np.mean(bucket_latency_mean[(10,10+R_1)]['P1_latency_mean'][-window_size:]),
                  np.mean(bucket_latency_mean[(10+R_1,10+2*R_1)]['P1_latency_mean'][-window_size:]) , np.mean(bucket_latency_mean[(10+2*R_1,10+3*R_1)]['P1_latency_mean'][-window_size:])
                  ,np.mean(bucket_latency_mean[(10+3*R_1,10+4*R_1)]['P1_latency_mean'][-window_size:]), np.mean(bucket_latency_mean[(10+4*R_1,float('inf'))]['P1_latency_mean'][-window_size:]))
            print('mean_task_P2_latency_by_bucket_51_10_100', np.mean(bucket_latency_mean[(10,10+R_1)]['P2_latency_mean'][-window_size:]),
                  np.mean(bucket_latency_mean[(10+R_1,10+2*R_1)]['P2_latency_mean'][-window_size:]) , np.mean(bucket_latency_mean[(10+2*R_1,10+3*R_1)]['P2_latency_mean'][-window_size:])
                  ,np.mean(bucket_latency_mean[(10+3*R_1,10+4*R_1)]['P2_latency_mean'][-window_size:]), np.mean(bucket_latency_mean[(10+4*R_1,float('inf'))]['P2_latency_mean'][-window_size:]))
            print('mean_task_P3_latency_by_bucket_51_10_100', np.mean(bucket_latency_mean[(10,10+R_1)]['P3_latency_mean'][-window_size:]),
                  np.mean(bucket_latency_mean[(10+R_1,10+2*R_1)]['P3_latency_mean'][-window_size:]) , np.mean(bucket_latency_mean[(10+2*R_1,10+3*R_1)]['P3_latency_mean'][-window_size:])
                  ,np.mean(bucket_latency_mean[(10+3*R_1,10+4*R_1)]['P3_latency_mean'][-window_size:]), np.mean(bucket_latency_mean[(10+4*R_1,float('inf'))]['P3_latency_mean'][-window_size:]))

            print('mean_task_P1_energy_by_bucket_51_10_100', np.mean(bucket_energy_mean[(10,10+R_1)]['P1_energy_mean'][-window_size:]),
                  np.mean(bucket_energy_mean[(10+R_1,10+2*R_1)]['P1_energy_mean'][-window_size:]) , np.mean(bucket_energy_mean[(10+2*R_1,10+3*R_1)]['P1_energy_mean'][-window_size:])
                  ,np.mean(bucket_energy_mean[(10+3*R_1,10+4*R_1)]['P1_energy_mean'][-window_size:]), np.mean(bucket_energy_mean[(10+4*R_1,float('inf'))]['P1_energy_mean'][-window_size:]))
            print('mean_task_P2_energy_by_bucket_51_10_100', np.mean(bucket_energy_mean[(10,10+R_1)]['P2_energy_mean'][-window_size:]),
                  np.mean(bucket_energy_mean[(10+R_1,10+2*R_1)]['P2_energy_mean'][-window_size:]) , np.mean(bucket_energy_mean[(10+2*R_1,10+3*R_1)]['P2_energy_mean'][-window_size:])
                  ,np.mean(bucket_energy_mean[(10+3*R_1,10+4*R_1)]['P2_energy_mean'][-window_size:]), np.mean(bucket_energy_mean[(10+4*R_1,float('inf'))]['P2_energy_mean'][-window_size:]))
            print('mean_task_P3_energy_by_bucket_51_10_100', np.mean(bucket_energy_mean[(10,10+R_1)]['P3_energy_mean'][-window_size:]),
                  np.mean(bucket_energy_mean[(10+R_1,10+2*R_1)]['P3_energy_mean'][-window_size:]) , np.mean(bucket_energy_mean[(10+2*R_1,10+3*R_1)]['P3_energy_mean'][-window_size:])
                  ,np.mean(bucket_energy_mean[(10+3*R_1,10+4*R_1)]['P3_energy_mean'][-window_size:]), np.mean(bucket_energy_mean[(10+4*R_1,float('inf'))]['P3_energy_mean'][-window_size:]))

            print('mean_data_P1_data_by_bucket_51_10_100', np.mean(bucket_data_mean[(10,10+R_1)]['P1_data_mean'][-window_size:]),
                  np.mean(bucket_data_mean[(10+R_1,10+2*R_1)]['P1_data_mean'][-window_size:]) , np.mean(bucket_data_mean[(10+2*R_1,10+3*R_1)]['P1_data_mean'][-window_size:])
                  ,np.mean(bucket_data_mean[(10+3*R_1,10+4*R_1)]['P1_data_mean'][-window_size:]), np.mean(bucket_data_mean[(10+4*R_1,float('inf'))]['P1_data_mean'][-window_size:]))
            print('mean_data_P2_data_by_bucket_51_10_100', np.mean(bucket_data_mean[(10,10+R_1)]['P2_data_mean'][-window_size:]),
                  np.mean(bucket_data_mean[(10+R_1,10+2*R_1)]['P2_data_mean'][-window_size:]) , np.mean(bucket_data_mean[(10+2*R_1,10+3*R_1)]['P2_data_mean'][-window_size:])
                  ,np.mean(bucket_data_mean[(10+3*R_1,10+4*R_1)]['P2_data_mean'][-window_size:]), np.mean(bucket_data_mean[(10+4*R_1,float('inf'))]['P2_data_mean'][-window_size:]))
            print('mean_data_P3_data_by_bucket_51_10_100', np.mean(bucket_data_mean[(10,10+R_1)]['P3_data_mean'][-window_size:]),
                  np.mean(bucket_data_mean[(10+R_1,10+2*R_1)]['P3_data_mean'][-window_size:]) , np.mean(bucket_data_mean[(10+2*R_1,10+3*R_1)]['P3_data_mean'][-window_size:])
                  ,np.mean(bucket_data_mean[(10+3*R_1,10+4*R_1)]['P3_data_mean'][-window_size:]), np.mean(bucket_data_mean[(10+4*R_1,float('inf'))]['P3_data_mean'][-window_size:]))


            print('all_tran',np.mean(epi_all_trans_latency[-window_size:]), 'all_comp',np.mean(epi_all_comp_latency[-window_size:]))
            
            print('all_tran_P1',np.mean(epi_all_trans_latency_P1[-window_size:]),\
                  'all_tran_P2',np.mean(epi_all_trans_latency_P2[-window_size:]),\
                      'all_tran_P3',np.mean(epi_all_trans_latency_P3[-window_size:]))
            
            print('all_comp_P1',np.mean(epi_all_comp_latency_P1[-window_size:]),\
                  'all_comp_P2',np.mean(epi_all_comp_latency_P2[-window_size:]),\
                 'all_comp_P3',np.mean(epi_all_comp_latency_P3[-window_size:]))
            
            print('all_data_P1',np.mean(epi_all_datasize_P1[-window_size:]),\
                  'all_data_P2',np.mean(epi_all_datasize_P2[-window_size:]),\
                 'all_data_P3',np.mean(epi_all_datasize_P3[-window_size:]))
            
            
            print('epi_comp_energy', np.mean(epi_comp_energy[-window_size:]))
            print('epi_comp_freq', np.mean(epi_comp_frequency[-window_size:]))
            print('epi_comp_latency', np.mean(epi_all_comp_latency[-window_size:]))
            
            if np.mean(scores_window)>=bench_score or i_episode % n_episodes==0:
                print('\nEnvironment solved in {:d} episodes!\tAverage Score: {:.4f}'.format(i_episode, mean_score))
                torch.save(agent.qnetwork_local.state_dict(), model_path)
                break

    return epi_scores,epi_task,epi_energy,epi_latency,epi_num_task, epi_task_P1,epi_task_P2,epi_task_P3,epi_latency_P1,epi_latency_P2,epi_latency_P3,epi_energy_P1,epi_energy_P2,epi_energy_P3,epi_num_task_P1,epi_num_task_P2,epi_num_task_P3, \
    epi_task_select_P1,epi_task_select_P2,epi_task_select_P3,mean_scores,mean_task,mean_energy,mean_latency,win_num_task, mean_tasks_P1,mean_tasks_P2,mean_tasks_P3, mean_energy_P1,\
                 mean_energy_P2, mean_energy_P3,mean_latency_P1,mean_latency_P2,mean_latency_P3,win_num_task_P1,win_num_task_P2,win_num_task_P3,epi_datasize,epi_datasize_P1,epi_datasize_P2,epi_datasize_P3,\
                 mean_datasize,mean_datasize_P1,mean_datasize_P2,mean_datasize_P3,epi_tran_latency_P1,epi_tran_latency_P2,\
                 epi_tran_latency_P3,epi_comp_latency_P1,epi_comp_latency_P2,epi_comp_latency_P3

dqnp_epi_scores,dqnp_epi_task,dqnp_epi_energy,dqnp_epi_latency,dqnp_epi_num_task,dqnp_epi_task_P1,dqnp_epi_task_P2,dqnp_epi_task_P3,dqnp_epi_latency_P1,dqnp_epi_latency_P2,dqnp_epi_latency_P3,\
dqnp_epi_energy_P1,dqnp_epi_energy_P2,dqnp_epi_energy_P3,dqnp_epi_num_task_P1,dqnp_epi_num_task_P2,dqnp_epi_num_task_P3,dqnp_epi_task_select_P1,dqnp_epi_task_select_P2,dqnp_epi_task_select_P3,\
dqnp_mean_scores,dqnp_mean_task,dqnp_mean_energy,dqnp_mean_latency,dqnp_win_num_task, dqnp_mean_tasks_P1,dqnp_mean_tasks_P2,dqnp_mean_tasks_P3,\
dqnp_mean_energy_P1,dqnp_mean_energy_P2,dqnp_mean_energy_P3,dqnp_mean_latency_P1,dqnp_mean_latency_P2,dqnp_mean_latency_P3 ,dqnp_win_num_task_P1, dqnp_win_num_task_P2,dqnp_win_num_task_P3,\
dqnp_epi_datasize,dqnp_epi_datasize_P1,dqnp_epi_datasize_P2,dqnp_epi_datasize_P3,dqnp_mean_datasize,dqnp_mean_datasize_P1,dqnp_mean_datasize_P2,dqnp_mean_datasize_P3,\
dqnp_epi_tran_latency_P1,dqnp_epi_tran_latency_P2,dqnp_epi_tran_latency_P3,dqnp_epi_comp_latency_P1,dqnp_epi_comp_latency_P2,\
dqnp_epi_comp_latency_P3= dqnp()

Episode 250	Average Score: -582.9342	meantaskP1: 0.5148	meantaskP2: 0.5114	meantaskP3: 0.4548	meanenergyP1:     8.1726	meanenergyP2: 16.9082	meanenergyP3: 12.6859 	meanlatencyP1: 0.0550 	meanlatencyP2: 0.0581 	meanlatencyP3:     0.0493 	meandatasize: 14914.9798 	meandatasizeP1: 14694.4080 	meandatasizeP2: 15054.6611 	meandatasizeP3: 15008.8582 	mean_tran_latencyP1: 0.0309      	mean_tran_latencyP2: 0.0359  	mean_tran_latencyP3: 0.0280  	mean_comp_latencyP1: 0.0241      	mean_comp_latencyP2: 0.0222 	mean_comp_latencyP3: 0.0213
mean_task_till_this_window 0.386308 T1:: 0.3892731237025264 T2:: 0.38535584599756145 T3:: 0.3835444277713923
task_select_P1 330.164 task_select_P2 333.16 task_select_P3 336.676
total_task_done_P1:: 128.38 total_task_done_P2:: 128.468 total_task_done_P3:: 129.46
mean_task_P1_ratio_by_bucket_51_10_100 0.9946535099579928 0.931198740053743 0.6877779286965044 0.3304658056876857 0.009376432521284638
mean_task_P2_ratio_by_bucket_51_10_100 0.9894101505747753 0.91727272890

Episode 1000	Average Score: 108.3071	meantaskP1: 0.6007	meantaskP2: 0.5921	meantaskP3: 0.5703	meanenergyP1:     5.8928	meanenergyP2: 10.6047	meanenergyP3: 11.5106 	meanlatencyP1: 0.0573 	meanlatencyP2: 0.0592 	meanlatencyP3:     0.0533 	meandatasize: 14862.4450 	meandatasizeP1: 14631.2948 	meandatasizeP2: 14940.2189 	meandatasizeP3: 14999.7102 	mean_tran_latencyP1: 0.0331      	mean_tran_latencyP2: 0.0361  	mean_tran_latencyP3: 0.0304  	mean_comp_latencyP1: 0.0242      	mean_comp_latencyP2: 0.0232 	mean_comp_latencyP3: 0.0229
mean_task_till_this_window 0.734756 T1:: 0.7398112396131076 T2:: 0.7375846417640921 T3:: 0.7271145098271233
task_select_P1 308.292 task_select_P2 324.696 task_select_P3 367.012
total_task_done_P1:: 228.072 total_task_done_P2:: 239.596 total_task_done_P3:: 267.088
mean_task_P1_ratio_by_bucket_51_10_100 0.9972023025453985 0.9598698798572045 0.8055779657371279 0.48436544468212933 0.06835336432695802
mean_task_P2_ratio_by_bucket_51_10_100 0.9957638003134164 0.95665411

Episode 1744	Average Score: 221.4667	meantaskP1: 0.8556	meantaskP2: 0.8118	meantaskP3: 0.7986	complatencyP1:             0.0255	complatencyP2: 0.0255	complatencyP3: 0.0248	meanlatencyP1: 0.0381	meanlatencyP2: 0.0363	meanlatencyP3:0.0322	meanlatreward:0.0796	meanengreward:0.1059

Episode 75	Average Score: -942.6496	meantaskP1: 0.2643	meantaskP2: 0.2371	meantaskP3: 0.1686	complatencyP1:             0.0069	complatencyP2: 0.0070	complatencyP3: 0.0070	meanlatencyP1: 0.0404	meanlatencyP2: 0.0429	meanlatencyP3:0.0300	meanlatreward:0.1205	meanengreward:1.5462

Episode 117	Average Score: 223.9963	meantaskP1: 0.5197	meantaskP2: 0.5284	meantaskP3: 0.5145	complatencyP1:             0.0104	complatencyP2: 0.0110	complatencyP3: 0.0104	meanlatencyP1: 0.0287	meanlatencyP2: 0.0322	meanlatencyP3:0.0352	meanlatreward:0.0318	meanengreward:0.0419

In [ ]:
import csv


# Combining data into a list of tuples
data = zip(range(1, len(dqnp_epi_scores) + 1), dqnp_epi_scores,dqnp_epi_task,dqnp_epi_energy,dqnp_epi_latency,dqnp_epi_num_task,dqnp_epi_task_P1,dqnp_epi_task_P2,
           dqnp_epi_task_P3,dqnp_epi_latency_P1,dqnp_epi_latency_P2,dqnp_epi_latency_P3,
           dqnp_epi_energy_P1,dqnp_epi_energy_P2,dqnp_epi_energy_P3,dqnp_epi_num_task_P1,dqnp_epi_num_task_P2,dqnp_epi_num_task_P3,
           dqnp_epi_datasize,dqnp_epi_datasize_P1,dqnp_epi_datasize_P2,dqnp_epi_datasize_P3,dqnp_epi_task_select_P1,dqnp_epi_task_select_P2,\
           dqnp_epi_task_select_P3,dqnp_epi_tran_latency_P1,dqnp_epi_tran_latency_P2,dqnp_epi_tran_latency_P3,\
                    dqnp_epi_comp_latency_P1,dqnp_epi_comp_latency_P2,dqnp_epi_comp_latency_P3)

# Writing data to a CSV file
with open("data_dqnp.csv", mode="w", newline="") as file:
    writer = csv.writer(file)
    writer.writerow(["Index","dqnp_epi_scores","dqnp_epi_task","dqnp_epi_energy","dqnp_epi_latency","dqnp_epi_num_task","dqnp_epi_task_P1","dqnp_epi_task_P2","dqnp_epi_task_P3",
                     "dqnp_epi_latency_P1","dqnp_epi_latency_P2","dqnp_epi_latency_P3","dqnp_epi_energy_P1","dqnp_epi_energy_P2","dqnp_epi_energy_P3","dqnp_epi_num_task_P1",
                     "dqnp_epi_num_task_P2","dqnp_epi_num_task_P3","dqnp_epi_datasize","dqnp_epi_datasize_P1","dqnp_epi_datasize_P2"
                     "dqnp_epi_datasize_P3","dqnp_epi_task_select_P1","dqnp_epi_task_select_P2","dqnp_epi_task_select_P3",\
                    "dqnp_epi_tran_latency_P1","dqnp_epi_tran_latency_P2","dqnp_epi_tran_latency_P3",\
                    "dqnp_epi_comp_latency_P1","dqnp_epi_comp_latency_P2","dqnp_epi_comp_latency_P3"])  # Writing header
    writer.writerows(data)  # Writing data rows




# 